In [ ]:
%pip install openicl
# Restart the kernel after the installation is completed

# 2. Using Different Language Models with OpenICL

In this chapter, we will show you how to use OpenICL to do in-context learning (ICL) with different language models. Mainly including [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), [FLAN-T5](https://arxiv.org/abs/2210.11416), [XGLM](https://arxiv.org/abs/2112.10668), OpenAI's [GPT-3](https://arxiv.org/abs/2005.14165) API and [OPT-175B](https://arxiv.org/abs/2205.01068) API.

## 2-1 Huggingface Library's Models

In this section, we will take GPT2, FLAN-T5, and XGLM as examples to show you how to use the models in the [huggingface library](https://huggingface.co/models) with OpenICL. Generally speaking, you only need to assign the corresponding name to the `model_name` parameter when declaring `Inferencer`, but we will still provide you with some specific examples.

### 2-1-1 GPT-2

This example can be found in [tutorial1](https://github.com/Shark-NLP/OpenICL/blob/main/examples/tutorials/openicl_tutorial1_getting_started.ipynb). But this time, we set `batch_size` for `TopkRetriever` and `PPLInference` to speed up. It can be noticed that the values ​​of the two `batch_size`(s) could be set to be different (`8` and `6`). That is because, at the beginning of retrieval and inference, the corresponding components will receive the complete dataset or the retrieval results for the entire test set, instead of processing the data in batches.

In [ ]:
from openicl import DatasetReader, PromptTemplate, TopkRetriever, PPLInferencer

# Define a DatasetReader, loading dataset from huggingface.
data = DatasetReader('gpt3mix/sst2', input_columns=['text'], output_column='label')

# SST-2 Template Example
tp_dict = {
     0: '</E>Positive Movie Review: </text>',
     1: '</E>Negative Movie Review: </text>' 
}
template = PromptTemplate(tp_dict, {'text' : '</text>'}, ice_token='</E>')

# TopK Retriever
retriever = TopkRetriever(data, ice_num=2, batch_size=8)

# Define a Inferencer
inferencer = PPLInferencer(model_name='distilgpt2', batch_size=6)

# Inference
predictions = inferencer.inference(retriever, ice_template=template, output_json_filename='sst2')
print(predictions)

### 2-1-2 XGLM

When it comes to machine translation, it is a good choice to use XGLM. But when using XGLM, we **don't suggest** to set `batch_size` in `GenInferencer`. (When calling the `model.generate` method of [huggingface transformers library](https://huggingface.co/docs/transformers/index), padding is needed if you want to input multiple pieces of data at a time. But we found in the test that if padding exists, the generation of XGLM will be affected). The code is as follows:

### 2-1-3 FLAN-T5